# Flotorch ADK - Complete Workflow Example

This notebook demonstrates the complete workflow using FlotorchADKAgent with FlotorchMemoryService.
This combines everything from previous examples into one complete solution.

## What you'll learn
- Using FlotorchADKAgent (from example 4)
- Adding FlotorchMemoryService (from example 3)
- Complete end-to-end workflow

## Requirements
- FLOTORCH_API_KEY environment variable
- FLOTORCH_BASE_URL environment variable


In [ ]:
%pip install --pre flotorch[adk]


In [ ]:
# Setup and imports
import asyncio
import os
import sys
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✓ Environment setup complete")


In [ ]:
# Import FlotorchADKAgent and FlotorchMemoryService
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.memory import FlotorchMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk import Runner
from google.genai import types

print("✓ All imports successful")


In [ ]:
# Configuration
APP_NAME = "flotorch_agent"
USER_ID = "flotorch_user_123"
MEMORY_PROVIDER = "memo-provider"

print(f"App Name: {APP_NAME}")
print(f"User ID: {USER_ID}")
print(f"Memory Provider: {MEMORY_PROVIDER}")


## Step 1: Create Memory Service

First, we create the FlotorchMemoryService that will store our conversations.


In [ ]:
# Create FlotorchMemoryService
memory_service = FlotorchMemoryService(
    api_key=os.getenv("FLOTORCH_API_KEY"),
    base_url=os.getenv("FLOTORCH_BASE_URL"),
    name=MEMORY_PROVIDER
)

print("✓ Memory service created")


In [ ]:
# Create FlotorchADKAgent with memory enabled
adk_client = FlotorchADKAgent(
    agent_name="flotorch_workflow_agent",
    enable_memory=True  # This is the key difference - memory is enabled
)

# Get the agent
agent = adk_client.get_agent()

print("✓ FlotorchADKAgent with memory created")


In [ ]:
# Setup runner with both agent and memory service
session_service = InMemorySessionService()
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service  # This connects memory to the runner
)

print("✓ Runner setup with memory service complete")


In [ ]:
# Helper function to send messages
def run_single_turn(query, session_id, user_id):
    """Send message to agent and get response."""
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    
    return "Sorry, I couldn't process that."

print("✓ Helper function ready")


## Test the Complete Workflow

Now let's test our complete setup with a conversation.


In [ ]:
# Create session and start conversation
async def start_conversation():
    session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"Complete workflow ready! Session: {session.id}")
    return session

# Start the conversation
session = await start_conversation()


In [ ]:
# Simple chat function
def chat(message):
    """Chat with the complete workflow agent."""
    response = run_single_turn(message, session.id, USER_ID)
    print(f"You: {message}")
    print(f"Flotorch AI: {response}")
    

# Have a conversation
print("=== Testing Complete Workflow ===")
chat("Hello! I'm John and I work as a data scientist.")


In [ ]:
# Continue the conversation
print("\n" + "-"*40)
chat("I'm working on a machine learning project about customer behavior.")


In [ ]:
# Store the conversation in memory
async def save_conversation():
    print("Saving conversation to memory...")
    
    completed_session = await runner.session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=session.id
    )
    
    await memory_service.add_session_to_memory(completed_session)
    print("✓ Conversation saved to memory!")

# Save the conversation
await save_conversation()


## Test Memory Recall

Let's create a new session to see if the agent remembers our previous conversation.


In [ ]:
# Create new session to test memory
async def test_memory():
    new_session = await runner.session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
    )
    
    print(f"New session created: {new_session.id}")
    
    # Test if agent remembers
    response = run_single_turn("What do you remember about me?", new_session.id, USER_ID)
    print(f"You: What do you remember about me?")
    print(f"Flotorch AI: {response}")

# Test memory recall
await test_memory()


## Summary

This complete workflow demonstrates:
- ✓ FlotorchADKAgent with memory enabled
- ✓ FlotorchMemoryService for storing conversations
- ✓ Memory recall across different sessions
- ✓ Complete integration of all components

**This combines everything from examples 1-4 into one complete solution!**

You can use `chat("your message")` to continue chatting with the complete workflow.
